# Clustering Crypto

In [125]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path
import numpy

### Fetching Cryptocurrency Data

In [126]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [127]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
df = df[["CoinName","Algorithm","IsTrading","ProofType","TotalCoinsMined","TotalCoinSupply"]]
df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [128]:
# Keep only cryptocurrencies that are trading
df = df[df.IsTrading == True]

In [129]:
# Keep only cryptocurrencies with a working algorithm
df = df[df.Algorithm != "N/A"]

In [130]:
# Remove the "IsTrading" column
df = df.drop(columns = ["IsTrading"])

In [131]:
# Remove rows with at least 1 null value
df = df.dropna()

In [132]:
# Remove rows with cryptocurrencies having no coins mined
df = df[df.TotalCoinsMined > 0]

In [133]:
# Drop rows where there are 'N/A' text values
df = df[(df.CoinName != "N/A")&(df.ProofType != "N/A")&(df.TotalCoinsMined != "N/A") & (df.TotalCoinSupply != "N/A")]

In [134]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
df_CoinName = df["CoinName"]
df_CoinName

0           42 Coin
2           404Coin
5         EliteCoin
7           Bitcoin
8          Ethereum
           ...     
1238         ZEPHYR
1242        Gapcoin
1245         Beldex
1246        Horizen
1247    BitcoinPlus
Name: CoinName, Length: 532, dtype: object

In [135]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
df = df.drop(columns=["CoinName"])
df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
1238,SHA-256,DPoS,2.000000e+09,2000000000
1242,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,CryptoNight,PoW,9.802226e+08,1400222610
1246,Equihash,PoW,7.296538e+06,21000000


In [136]:
# Create dummy variables for text features
df["TotalCoinSupply"] = pd.to_numeric(df["TotalCoinSupply"])
df_cleaned = pd.get_dummies(df)
df_cleaned = df_cleaned.drop(columns=["Algorithm_536","ProofType_Pos"]) #makes 536 base algorithm and Pos base ProofType

In [137]:
# Standardize data
df_scaled = StandardScaler().fit_transform(df_cleaned)
df_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Reducing Dimensions Using PCA

In [138]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(df_scaled)

In [139]:
# Create a DataFrame with the principal components data
df_crypto_pca = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2","PC 3"]
)

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [140]:
k = list(range(1, 11))

# Calculate the inertia for the range of k values
def inertia(k,df):
    inertia = []
    for i in k:
        km = KMeans(n_clusters=i, random_state=0)
        km.fit(df)
        inertia.append(km.inertia_)
    return inertia

inertia = inertia(k,df_crypto_pca)

# Create the Elbow Curve using hvPlot
def elbow_curve(k,inertia):
    elbow_data = {"k": k, "inertia": inertia}
    df_elbow = pd.DataFrame(elbow_data)
    return df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

elbow_graph = elbow_curve(k,inertia)
elbow_graph

C:\Users\hmast\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [141]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(df_crypto_pca)

# Predict clusters
predictions = model.predict(df_crypto_pca)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
df_crypto_pca["class"] = model.labels_
df_crypto_pca.head()


,PC 1,PC 2,PC 3,class
0,-0.336700,1.011433,-0.620282,3
1,-0.320036,1.011591,-0.620684,3
2,2.304223,1.679930,-0.684773,3
3,-0.142543,-1.293811,0.221918,0
4,-0.150774,-2.010791,0.421658,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [142]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    df_crypto_pca,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="class",
    symbol="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [143]:
# Table with tradable cryptos
tradable_cryptos = df
df["Coin Name"] = df_CoinName
df["Class"] = model.labels_
tradable_cryptos.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,Coin Name,Class
0,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,42 Coin,3
2,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,404Coin,3
5,X13,PoW/PoS,2.927942e+10,3.141593e+11,EliteCoin,3
7,SHA-256,PoW,1.792718e+07,2.100000e+07,Bitcoin,0
8,Ethash,PoW,1.076842e+08,0.000000e+00,Ethereum,0


In [144]:
# Print the total number of tradable cryptocurrencies
len(tradable_cryptos)

532

#### Scatter Plot with Tradable Cryptocurrencies

In [145]:
# Scale data to create the scatter plot
#scaling done by turning axis into log

In [148]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
tradable_cryptos.hvplot.scatter(x="TotalCoinsMined",y="TotalCoinSupply", logx=True,logy=True)

:Scatter   [TotalCoinsMined]   (TotalCoinSupply)